# 2. Gathering Play Information
Later in my analysis, I will need some variables identifying important play information. I decided to process all the variables beforehand and save them to a .csv file for future use. This will save me from recalculating them multiple times in the future.

## 2.1 Imports

In [1]:
library(tidyverse)
library(here)

source(here("R", "00_source.R"))

puntPlays <- read.csv(here("data", "punt_plays.csv"))

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.8
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

here() starts at C:/Users/Hunter



## 2.2 Create New Dataframe to Store Important Play Information
* **gameId:** Game identifier, unique (numeric)
* **playId:** Play identifier, not unique across games (numeric)
* **snapFrame:** Frame in tracking data that ball was snapped (numeric)
* **catchFrame:** Frame in tracking data that ball was caught (numeric)
* **kickDir:** Kick direction from kicking team's perspective (character)
  + Possible values:
    + **L:** Left
    + **R:** Right
    + **C:** Center
* **returnYds:** Yards gained by return team (numeric)
* **specialTeamsResult:** Special teams outcome of play (character)

In [2]:
puntPlaysID <- unique(puntPlays[c('gameId', 'playId')])

puntPlayInfo <- data.frame(gameId = puntPlaysID$gameId,
                           playId = puntPlaysID$playId,
                           snapFrame = numeric(nrow(puntPlaysID)),
                           catchFrame = numeric(nrow(puntPlaysID)),
                           kickDir = character(nrow(puntPlaysID)),
                           returnYds = numeric(nrow(puntPlaysID)),
                           specialTeamsResult = character(nrow(puntPlaysID)))

## 2.3 Fill *snapFrame*, *catchFrame*, and *ballCatchRow* Columns
This is possible by using the *event* variable and checking which frame the 'ball_snap' event occurred. However, some plays don't have this event listed. Below we can see that almost every play has the ball being snapped at the eleventh frame. If the 'ball_snap' event doesn't exist, I will assume it happened at frame eleven. For filling the *catchFrame* column, I can check for events such as 'fair_catch', 'punt_received', or 'punt_land'. If none of these events exist, I can estimate it by adding the *snapFrame*, *operationTime*, and *hangTime* together. 

In [3]:
ballSnapFrame <- puntPlays$frameId[puntPlays$event == 'ball_snap' &
                                   puntPlays$displayName == 'football']
table(ballSnapFrame)

ballSnapFrame
   1    2    3    4    5    7    8    9   10   11 
   3    1    2    3    2    2    5    3   23 2055 

In [4]:
for (i in 1:nrow(puntPlayInfo)) {
  play <- puntPlays %>% filter(gameId == puntPlayInfo$gameId[i] & 
                                 playId == puntPlayInfo$playId[i])
  
  puntPlayInfo$snapFrame[i] <- play$frameId[which(play$event == 'ball_snap')][1]
  
  if (is.na(puntPlayInfo$snapFrame[i])) {
    puntPlayInfo$snapFrame[i] <- 11
  }
  
  catchFrameCands <- play$frameId[which(play$event == 'fair_catch' | 
                                        play$event == 'punt_received' |
                                        play$event == 'punt_land')]
  if(length(catchFrameCands) > 0) {
    puntPlayInfo$catchFrame[i] <- max(catchFrameCands)
  } else {
    puntPlayInfo$catchFrame[i] <- puntPlayInfo$snapFrame[i] + 
                                  ceiling(play$operationTime[1] * 10) + 
                                  ceiling(play$hangTime[1] * 10)
  }
}

In [5]:
head(puntPlayInfo)

,gameId,playId,snapFrame,catchFrame,kickDir,returnYds,specialTeamsResult
,<int>,<int>,<dbl>,<dbl>,<chr>,<dbl>,<chr>
1,2018123000,892,11,76,,0,
2,2018123000,2502,11,75,,0,
3,2018123000,2975,11,76,,0,
4,2018123000,3106,11,94,,0,
5,2018123000,4148,11,76,,0,
6,2018123001,1223,11,94,,0,


## 2.4 Fill *kickDir* Column

In [6]:
for (i in 1:nrow(puntPlayInfo)) {
  play <- puntPlays %>% filter(gameId == puntPlayInfo$gameId[i] & 
                               playId == puntPlayInfo$playId[i])
  
  puntPlayInfo$kickDir[i] <- play$kickDirectionActual[1]
}

In [7]:
head(puntPlayInfo)

,gameId,playId,snapFrame,catchFrame,kickDir,returnYds,specialTeamsResult
,<int>,<int>,<dbl>,<dbl>,<chr>,<dbl>,<chr>
1,2018123000,892,11,76,R,0,
2,2018123000,2502,11,75,C,0,
3,2018123000,2975,11,76,R,0,
4,2018123000,3106,11,94,L,0,
5,2018123000,4148,11,76,L,0,
6,2018123001,1223,11,94,C,0,


## 2.5 Fill *returnYds* and *specialTeamsResult* Columns

In [8]:
returnYds <- numeric(nrow(puntPlayInfo))
for (i in 1:nrow(puntPlayInfo)) {
  play <- puntPlays %>% filter(gameId == puntPlayInfo$gameId[i] & 
                               playId == puntPlayInfo$playId[i])
  
  puntPlayInfo$specialTeamsResult[i] <- play$specialTeamsResult[1]
  returnYds[i] <- play$kickReturnYardage[1]
}
puntPlayInfo$returnYds <- returnYds

In [9]:
head(puntPlayInfo)

,gameId,playId,snapFrame,catchFrame,kickDir,returnYds,specialTeamsResult
,<int>,<int>,<dbl>,<dbl>,<chr>,<dbl>,<chr>
1,2018123000,892,11,76,R,0,Fair Catch
2,2018123000,2502,11,75,C,37,Return
3,2018123000,2975,11,76,R,0,Fair Catch
4,2018123000,3106,11,94,L,8,Return
5,2018123000,4148,11,76,L,1,Return
6,2018123001,1223,11,94,C,0,Return


## 2.6 Write to .csv

In [10]:
write.csv(puntPlayInfo, here("data", "punt_play_info.csv"), row.names = FALSE)